In [118]:
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


In [119]:
CLASS_NAMES = ['Distance_Driver', 'Control_Driver', 'Putt_&_Approach', 'Mid_Range']

In [120]:
disc_data=pd.read_csv("C:/Users/idansim/Dropbox/Work/Python_tutorials/Disc_Golf/disc-data.csv")

In [121]:
# Replace spaces in column names
disc_data.columns = disc_data.columns.str.replace(' ', '_')
disc_data.columns = disc_data.columns.str.replace('[()%]', '', regex=True)

# Replace spaces in string values
disc_data = disc_data.apply(lambda col: col.str.replace(' ', '_') if col.dtype == 'object' else col)

disc_data = disc_data.drop('MOLD', axis=1)

In [122]:
disc_data['BEAD'] = disc_data['BEAD'].replace({'Yes': 1, 'No': 0})

C:\Users\idansim\AppData\Local\Temp\ipykernel_32028\3057763355.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  disc_data['BEAD'] = disc_data['BEAD'].replace({'Yes': 1, 'No': 0})


In [123]:
disc_data

,DISC_TYPE,SPEED,GLIDE,TURN,FADE,STABILITY,DIAMETER_cm,HEIGHT_cm,RIM_DEPTH_cm,RIM_WIDTH_cm,INSIDE_RIM_DIAMETER_cm,RIM_DEPTH_/_DIAMETER_RATION_,RIM_CONFIGURATION,BEAD
0,Putt_&_Approach,1.0,2.0,0.0,1.0,1.0,21.3,1.9,1.4,0.8,19.6,6.6,90.75,0
1,Putt_&_Approach,1.0,7.0,-1.0,0.0,-1.0,21.9,2.2,1.6,0.8,20.3,7.3,89.25,0
2,Putt_&_Approach,1.0,1.0,0.0,2.0,2.0,21.1,2.0,1.4,0.9,19.3,6.6,87.00,0
3,Putt_&_Approach,1.0,2.0,0.0,0.0,0.0,21.2,2.4,1.9,0.7,19.8,9.0,98.25,0
4,Putt_&_Approach,1.0,3.0,0.0,0.5,0.5,21.2,2.0,1.7,0.9,19.4,8.0,85.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,Distance_Driver,14.5,5.0,-0.5,3.0,2.5,21.1,1.8,1.2,2.5,16.0,5.7,27.00,0
1171,Distance_Driver,14.5,5.0,0.0,3.0,3.0,21.0,1.8,1.2,2.5,16.0,5.7,27.75,0
1172,Distance_Driver,14.5,5.5,-2.0,2.0,0.0,21.1,2.1,1.2,2.5,15.9,5.7,27.00,0
1173,Distance_Driver,14.5,3.5,0.0,4.0,4.0,21.1,1.8,1.2,2.5,16.0,5.7,27.25,0


In [124]:
disc_data['BEAD'] = disc_data['BEAD'].replace({'Yes': 1, 'No': 0})

scaler = MinMaxScaler()
numerical_columns = disc_data.select_dtypes(include=['number']).columns
disc_data[numerical_columns] = scaler.fit_transform(disc_data[numerical_columns])



# Initialize LabelEncoder
encoder = LabelEncoder()
# Apply label encoding
disc_data['DISC_TYPE'] = encoder.fit_transform(disc_data['DISC_TYPE'])


In [125]:
# Sample a tenth of the data to use as test data
disc_test = disc_data.sample(n=(disc_data.shape[0]//10), random_state=42)
disc_train = disc_data.drop(disc_test.index)
disc_train.reset_index(drop=True, inplace=True)
disc_test.reset_index(drop=True, inplace=True)

disc_train_type = disc_train.pop('DISC_TYPE')
disc_test_type = disc_test.pop('DISC_TYPE')

assert disc_test.shape[0] == disc_data.shape[0]//10
assert disc_train.shape[0] == disc_data.shape[0]-disc_test.shape[0]

disc_train

,SPEED,GLIDE,TURN,FADE,STABILITY,DIAMETER_cm,HEIGHT_cm,RIM_DEPTH_cm,RIM_WIDTH_cm,INSIDE_RIM_DIAMETER_cm,RIM_DEPTH_/_DIAMETER_RATION_,RIM_CONFIGURATION,BEAD
0,0.0,0.285714,0.714286,0.166667,0.454545,0.09375,0.3750,0.250000,0.10,0.536232,0.340426,0.896194,0.0
1,0.0,1.000000,0.571429,0.000000,0.272727,0.28125,0.5625,0.416667,0.10,0.637681,0.489362,0.875433,0.0
2,0.0,0.142857,0.714286,0.333333,0.545455,0.03125,0.4375,0.250000,0.15,0.492754,0.340426,0.844291,0.0
3,0.0,0.285714,0.714286,0.000000,0.363636,0.06250,0.6875,0.666667,0.05,0.565217,0.851064,1.000000,0.0
4,0.0,0.428571,0.714286,0.083333,0.409091,0.06250,0.4375,0.500000,0.15,0.507246,0.638298,0.816609,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1.0,0.714286,0.642857,0.500000,0.590909,0.03125,0.3125,0.083333,0.95,0.014493,0.148936,0.013841,0.0
1054,1.0,0.714286,0.714286,0.500000,0.636364,0.00000,0.3125,0.083333,0.95,0.014493,0.148936,0.024221,0.0
1055,1.0,0.785714,0.428571,0.333333,0.363636,0.03125,0.5000,0.083333,0.95,0.000000,0.148936,0.013841,0.0
1056,1.0,0.500000,0.714286,0.666667,0.727273,0.03125,0.3125,0.083333,0.95,0.014493,0.148936,0.017301,0.0


In [126]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


In [127]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in disc_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SPEED', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GLIDE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TURN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FADE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='STABILITY', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIAMETER_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HEIGHT_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RIM_DEPTH_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RIM_WIDTH_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INSIDE_RIM_DIAMETER_cm', shape=(1,), default_value=None, dtype=tf.float3

In [128]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\idansim\\AppData\\Local\\Temp\\tmpji8p34xe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [129]:
classifier.train(
    input_fn=lambda: input_fn(disc_train, disc_train_type, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\idansim\AppData\Local\Temp\tmpji8p34xe\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use t

In [130]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(disc_test, disc_test_type, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-07-30T18:44:19
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\idansim\AppData\Local\Temp\tmpji8p34xe\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.25325s
INFO:tensorflow:Finished evaluation at 2024-07-30-18:44:19
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7777778, average_loss = 1.0525957, global_step = 5000, loss = 1.0525957
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\idansim\AppData\Local\Temp\tmpji8p34xe\model.ckpt-5000

Test set accuracy: 0.778

